<a href="https://colab.research.google.com/github/sibyjackgrove/SolarPV-DER-simulation-utility/blob/master/examples/PV-DER_usage_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Accessing and updating model parameters

### Import classes
Import classes neccessary to define instances of the PV-DER model, events, simulation, and results.

In [1]:
#Import PV-DER modules
from pvder.DER_components_single_phase import SolarPV_DER_SinglePhase
from pvder.DER_components_three_phase  import SolarPV_DER_ThreePhase
from pvder.grid_components import BaseValues
from pvder.dynamic_simulation import DynamicSimulation
from pvder.simulation_events import SimulationEvents
from pvder.simulation_utilities import SimulationUtilities,SimulationResults
from pvder import utility_functions

import math
import logging

#To show interactive plot in the notebook cell
%matplotlib inline    

### Specify voltage values
Specify voltage values for non-standalone model.

In [2]:
scaler = 0.835
Va = (.50+0j)*BaseValues.Vbase*scaler
Vb = (-.25-.43301270j)*BaseValues.Vbase*scaler
Vc = (-.25+.43301270j)*BaseValues.Vbase*scaler
Vrms = abs(Va)/math.sqrt(2)
print('Va:{},Vrms:{}'.format(abs(Va),Vrms))
logging.debug('test')

Va:208.75,Vrms:147.6085405726918


### Create objects required to simulate either single phase or three phase PV-DER

In [3]:
events1 = SimulationEvents(verbosity = 'INFO')
PV_DER1 = SolarPV_DER_ThreePhase(events=events1,Sinverter_rated = 50.0e3,Vrms_rated = Vrms, #175
                                     gridVoltagePhaseA = Va,
                                     gridVoltagePhaseB = Vb,
                                     gridVoltagePhaseC = Vc,
                                     gridFrequency=2*math.pi*60.0,
                                     standAlone = False,STEADY_STATE_INITIALIZATION=True,verbosity = 'INFO')   
sim1 = DynamicSimulation(PV_model=PV_DER1,events = events1,verbosity = 'INFO',solver_type='odeint')

INFO:SolarPV_DER_ThreePhase:PVDER-3ph_1:Instance created with parameter ID: 50; Specifications - Srated:50.0 kVA, Vrms:147.6 V ,Steady state:True,LVRT Enable:True, LVRT Instantaneous trip:False


## Create and use a user defined parameter dictionary
### 1. Show existing parameter dictionaries using show_parameter_dictionaries() and show_parameter_types().


In [4]:
PV_DER1.show_parameter_dictionaries()
PV_DER1.show_parameter_types()

-----Parameter dictionary: Parameter IDs-----
module_parameters : 10,50,250
inverter_ratings : 50,250
circuit_parameters : 50,250
controller_parameters : 50,250,250_1
steadystate_values : 50,250
-----Parameter dictionary: Parameter types-----
module_parameters : Np,Ns,Vdcmpp0,Vdcmpp_min,Vdcmpp_max
inverter_ratings : Srated,Varated,Vdcrated,Ioverload
circuit_parameters : Rf_actual,Lf_actual,C_actual,Z1_actual
controller_parameters : scale_Kp_GCC,scale_Ki_GCC,scale_Kp_DC,scale_Ki_DC,scale_Kp_Q,scale_Ki_Q,wp
steadystate_values : maR0,maI0,iaR0,iaI0


### 2. Get parameter values using get_parameter_values()

In [5]:
DER_parameters = PV_DER1.get_parameter_dictionary(parameter_type='all',parameter_ID='50')

{   'C_actual': 0.0003,
    'Ioverload': 1.5,
    'Lf_actual': 2.5e-05,
    'Np': 11,
    'Ns': 735,
    'Rf_actual': 0.002,
    'Srated': 50000.0,
    'Varated': 245.0,
    'Vdcmpp0': 550.0,
    'Vdcmpp_max': 650.0,
    'Vdcmpp_min': 520.0,
    'Vdcrated': 550.0,
    'Z1_actual': (0.0019+0.0561j),
    'iaI0': 0.001,
    'iaR0': 1.0,
    'maI0': 0.0,
    'maR0': 0.89,
    'scale_Ki_DC': 0.05,
    'scale_Ki_GCC': 0.05,
    'scale_Ki_Q': 0.05,
    'scale_Kp_DC': 0.05,
    'scale_Kp_GCC': 0.05,
    'scale_Kp_Q': 0.05,
    'wp': 200000.0}


### 3. Select name of new parameter dictionary (e.g. 'my_DER') and source parameter ID (e.g. '50').
### 4. Create and initialize new parameter dictionary using initialize_parameter_dict().

In [6]:
PV_DER1.initialize_parameter_dict(parameter_ID='my_DER',source_parameter_ID='50')

INFO:SolarPV_DER_ThreePhase:PVDER-3ph_1:Created and initialized new parameter dicitonary my_DER with source dictionary 50.


### 5. Update the new parameter dictionary with desired values (e.g. {'Np':5} in 'module_parameters' and {'C_actual':5} in 'circuit_parameters') using update_parameter_dict().

In [7]:
PV_DER1.update_parameter_dict(parameter_ID='my_DER',parameter_type='module_parameters',parameter_dict={'Np':5})
PV_DER1.update_parameter_dict(parameter_ID='my_DER',parameter_type='circuit_parameters',parameter_dict={'C_actual':0.0005})

### 6. Update the PV-DER model with parameters from the new parameter dictionary using modify_DER_parameters()

In [8]:
PV_DER1.modify_DER_parameters(parameter_ID='my_DER')

INFO:SolarPV_DER_ThreePhase:PVDER-3ph_1:PV-DER parameters updated with parameters from  parameter dictionary my_DER!


### 7. Save and load parameter dictionaries using  save_parameter_dictionary() and load_parameter_dictionary()

In [9]:
file_name = PV_DER1.save_parameter_dictionary(parameter_ID='my_DER',save_format='pickle')

INFO:SolarPV_DER_ThreePhase:PVDER-3ph_1:Saved all the parameter dicitonaries as a pickle file in my_DER.pkl.


In [10]:
_=PV_DER1.load_parameter_dictionary(file_name='my_DER.pkl')

Read following parameter dictionary from my_DER.pkl:
{   'C_actual': 0.0005,
    'Ioverload': 1.5,
    'Lf_actual': 2.5e-05,
    'Np': 5,
    'Ns': 735,
    'Rf_actual': 0.002,
    'Srated': 50000.0,
    'Varated': 245.0,
    'Vdcmpp0': 550.0,
    'Vdcmpp_max': 650.0,
    'Vdcmpp_min': 520.0,
    'Vdcrated': 550.0,
    'Z1_actual': (0.0019+0.0561j),
    'iaI0': 0.001,
    'iaR0': 1.0,
    'maI0': 0.0,
    'maR0': 0.89,
    'scale_Ki_DC': 0.05,
    'scale_Ki_GCC': 0.05,
    'scale_Ki_Q': 0.05,
    'scale_Kp_DC': 0.05,
    'scale_Kp_GCC': 0.05,
    'scale_Kp_Q': 0.05,
    'wp': 200000.0}


## Specify run time flags, simulation time, and run simulation 

In [11]:
%%time
sim1.jacFlag = True
sim1.DEBUG_SIMULATION = False
sim1.PER_UNIT = True
sim1.DEBUG_SOLVER  = False
sim1.tStop = 10.0
sim1.tInc = 1/120.# 0.001
sim1.run_simulation()
PV_DER1.validate_model()
PV_DER1.show_PV_DER_states('voltage')
PV_DER1.show_PV_DER_states('current')
PV_DER1.show_PV_DER_states('power')
PV_DER1.show_PV_DER_states('duty cycle')

sim_1:Simulation started at 0.0 s and will end at 10.0 s
sim_1:Simulation was completed in 00:00:00
Active power output error:0.0000
Reactive power output error:0.0000
Inverter filter active power loss error:0.0000
Inverter filter reactive power loss error:0.0000

______PVDER-3ph_1 - Voltage_____
Vdc:550.00
Vta:208.88+0.63j V
Vtb:-103.90-181.21j V,Vtb:-104.98+180.58j V
Vtn:0.00-0.00j V
Va:208.75+0.00j V
Vb:-104.38-180.78j V,Vc:-104.38+180.78j V
Vn:0.00-0.00j V
Vtrms:147.70 V
Vpccrms:147.61 V

______PVDER-3ph_1 - Current_____
ia:66.38+0.00j A
ib:-33.19-57.48j A,ic:-33.19+57.48j A
In:0.00+0.00j A
Irms:46.94 V

______PVDER-3ph_1 - Power_____
Ppv:20797.7 W
S:20797.7+62.3j VA
S_PCC:20784.5-0.0j VA

______PVDER-3ph_1 - Duty cycle_____
ma:0.76+0.00j
mb:-0.38-0.66j,mc:-0.38+0.66j
m0:0.00-0.00j
Wall time: 253 ms


C:\ProgramData\Anaconda3\lib\site-packages\scipy\integrate\odepack.py:250: ODEintWarning: Integration successful.
  warnings.warn(warning_msg, ODEintWarning)


### Get trajectories for current, voltage, and power as a dictionary

In [12]:
trajectories = sim1.get_trajectories()